In [91]:


import re
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from string import punctuation
import pickle
# from sklearn.feature_extraction.text import TfidfVectorizer

In [92]:
import requests
import pandas as pd

import numpy as np
import datetime
from sqlalchemy import create_engine

engine = create_engine(
    'postgresql://postgres:<password here>@tweets-sentiment.cwbdzmao4m7w.us-east-1.rds.amazonaws.com/tweets_sentiment_db')

df = pd.read_sql('SELECT * FROM tweets2', engine, index_col="id")
print(df.describe())

                      created_at  \
count                      31774   
unique                     28094   
top     2022-01-20T17:03:08.000Z   
freq                           6   

                                                     text  
count                                               31774  
unique                                               9740  
top     RT @estrellasadie: Bumped into conspiracy neig...  
freq                                                 1822  


In [93]:
df.describe()

,created_at,text
count,31774,31774
unique,28094,9740
top,2022-01-20T17:03:08.000Z,RT @estrellasadie: Bumped into conspiracy neig...
freq,6,1822


In [94]:
df.dtypes

created_at    object
text          object
dtype: object

In [97]:
df.head()

,created_at,text
id,,
1484408830444597250,2022-01-21T06:13:38.000Z,RT @estrellasadie: Bumped into conspiracy neig...
1484408727423922177,2022-01-21T06:13:14.000Z,RT @vademocrats: .@GovernorVA is putting polit...
1484408723737038848,2022-01-21T06:13:13.000Z,RT @estrellasadie: Bumped into conspiracy neig...
1484408717286150145,2022-01-21T06:13:11.000Z,@JoyceLauNews Thank goodness the school distri...
1484408650135314432,2022-01-21T06:12:55.000Z,RT @RepBalderson: Speaker Pelosi’s Capitol Hil...


In [98]:
def tag2type(tag):

    if tag.startswith('NN'):
        return 'n'
    elif tag.startswith('VB'):
        return 'v'
    else:
        return 'a'

stopwords1=stopwords.words('english')
punctuations_list = punctuation

def cleaning_URLs(data):
        return re.sub('(www.[^\s]+)|(http?://[^\s]+)|(https?://[^\s]+)','',data)
def clean_text(data):
    data = data.lower()

    data = cleaning_URLs(data)

    data =" ".join([word for word in str(data).split() if word not in stopwords1])

    translator = str.maketrans('', '', punctuations_list)
    data = data.translate(translator)
    data = re.sub('[0-9]+','',data)
    tweet_tokenizer = TweetTokenizer(
    preserve_case = True,
    reduce_len    = False,
    strip_handles = False)

    tweet=data

    tokens = tweet_tokenizer.tokenize(tweet)

    data = tokens


    tags = pos_tag(data)
    lemmatizer = WordNetLemmatizer()



    data = [lemmatizer.lemmatize(t[0], tag2type(t[1])) for t in tags]

    data = ' '.join(data)

    data = [data]

    return data
print(clean_text('http://www.sadsaas.com good'))

['good']


In [99]:
file = open('LRmodel.pickle', 'rb')
info = pickle.load(file)
file.close()

file2 = open('vectorizer.pickle', 'rb')
info2 = pickle.load(file2)
file2.close()

In [100]:

data = info2.transform(clean_text('http://www.sadsaas.com bad'))



In [101]:
info.predict(data)

array([0])

In [102]:
df.shape

(31774, 2)

In [103]:
import datetime
datetime.datetime.now()

datetime.datetime(2022, 1, 21, 1, 34, 20, 99095)

In [104]:
df['sentiment']=df['text'].apply(lambda x: info.predict(info2.transform(clean_text(x))))
df.head()

,created_at,text,sentiment
id,,,
1484408830444597250,2022-01-21T06:13:38.000Z,RT @estrellasadie: Bumped into conspiracy neig...,[0]
1484408727423922177,2022-01-21T06:13:14.000Z,RT @vademocrats: .@GovernorVA is putting polit...,[1]
1484408723737038848,2022-01-21T06:13:13.000Z,RT @estrellasadie: Bumped into conspiracy neig...,[0]
1484408717286150145,2022-01-21T06:13:11.000Z,@JoyceLauNews Thank goodness the school distri...,[1]
1484408650135314432,2022-01-21T06:12:55.000Z,RT @RepBalderson: Speaker Pelosi’s Capitol Hil...,[1]


In [ ]:
df.reset_index(inplace=True)
df['sentiment']=df['sentiment'].apply(lambda x: x[0])

In [2]:
import numpy as np
q = np.int64(123)
print(q)

123


In [5]:
q = int(q)

In [7]:
q

123

In [6]:
type(q)

int

In [117]:
datetime.datetime.now()

datetime.datetime(2022, 1, 21, 1, 40, 50, 526279)

In [118]:

df.to_sql('tweets', engine, index=False, if_exists="append")

In [119]:
datetime.datetime.now()

datetime.datetime(2022, 1, 21, 1, 41, 2, 79362)